In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
import os
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [0]:
!wget https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-vrd.csv

--2019-07-15 05:23:15--  https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-vrd.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41928675 (40M) [application/octet-stream]
Saving to: ‘challenge-2019-train-vrd.csv.2’

challenge-2019-trai 100%[===================>]  39.99M   188MB/s    in 0.2s    

2019-07-15 05:23:15 (188 MB/s) - ‘challenge-2019-train-vrd.csv.2’ saved [41928675/41928675]



In [0]:
df=pd.read_csv("challenge-2019-train-vrd.csv")

In [0]:
df=df[df['RelationshipLabel']!='is']

In [0]:
df.sort_values(by=['ImageID'], inplace=True)

df = df.reset_index(drop=True)
df.head(5)

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,000060e3121c7305,/m/01mzpv,/m/04bcr3,0.36875,0.538750,0.623243,0.999063,0.22875,0.696875,0.465792,0.926898,at
1,000060e3121c7305,/m/01mzpv,/m/01y9k5,0.36875,0.538750,0.623243,0.999063,0.23125,0.712500,0.649485,0.970947,at
2,0000615b5a80f660,/m/01mzpv,/m/078n6m,0.00375,0.166250,0.663333,0.941667,0.00000,0.694375,0.778333,0.999167,at
3,0000615b5a80f660,/m/09tvcd,/m/078n6m,0.34875,0.400625,0.760000,0.913333,0.00000,0.694375,0.778333,0.999167,on
4,0000615b5a80f660,/m/01mzpv,/m/078n6m,0.00000,0.114375,0.926667,0.999167,0.00000,0.694375,0.778333,0.999167,at


In [0]:
df['RelationshipLabel'].value_counts()


at                111493
on                 31604
holds              20986
plays               8932
interacts_with      3756
inside_of           2392
wears                836
hits                 593
under                 34
Name: RelationshipLabel, dtype: int64

In [0]:
ids=df.ImageID.unique()
len(ids)

57187

In [0]:
!wget https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-vrd-bbox.csv

--2019-07-15 05:23:21--  https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-vrd-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.187.128, 2404:6800:4008:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.187.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212551504 (203M) [application/octet-stream]
Saving to: ‘challenge-2019-train-vrd-bbox.csv.2’

challenge-2019-trai 100%[===================>] 202.70M   190MB/s    in 1.1s    

2019-07-15 05:23:22 (190 MB/s) - ‘challenge-2019-train-vrd-bbox.csv.2’ saved [212551504/212551504]



In [0]:
df2=pd.read_csv("challenge-2019-train-vrd-bbox.csv")

In [0]:
df2.sort_values(by=['ImageID'], inplace=True)

In [0]:
df2=df2[df2["ImageID"].isin(ids)]
df2=df2.reset_index(drop=True)


In [0]:
df2=df2[['ImageID','LabelName','XMin','XMax','YMin','YMax']]

In [0]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419453 entries, 0 to 419452
Data columns (total 6 columns):
ImageID      419453 non-null object
LabelName    419453 non-null object
XMin         419453 non-null float64
XMax         419453 non-null float64
YMin         419453 non-null float64
YMax         419453 non-null float64
dtypes: float64(4), object(2)
memory usage: 19.2+ MB


In [0]:
df2.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
1,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947
2,000060e3121c7305,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
3,000060e3121c7305,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
4,000060e3121c7305,/m/04bcr3,0.630000,0.996875,0.685098,0.999063


In [0]:
dfx= pd.merge(df2,df2, left_on='ImageID', right_on='ImageID', how='outer')
rem=dfx[ (dfx['LabelName_x'] == dfx['LabelName_y']) & (dfx['XMin_x'] == dfx['XMin_y']) & (dfx['XMax_x'] == dfx['XMax_y'])& (dfx['YMin_x'] == dfx['YMin_y']) & (dfx['XMin_x'] == dfx['XMin_y'])]

In [0]:
dfx.drop(rem.index)

,ImageID,LabelName_x,XMin_x,XMax_x,YMin_x,YMax_x,LabelName_y,XMin_y,XMax_y,YMin_y,YMax_y
1,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/01y9k5,0.231250,0.712500,0.649485,0.970947
2,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
3,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
4,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/04bcr3,0.630000,0.996875,0.685098,0.999063
5,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
7,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
8,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
9,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/04bcr3,0.630000,0.996875,0.685098,0.999063
10,000060e3121c7305,/m/01y9k5,0.629375,0.999375,0.691659,0.997188,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
11,000060e3121c7305,/m/01y9k5,0.629375,0.999375,0.691659,0.997188,/m/01y9k5,0.231250,0.712500,0.649485,0.970947


In [0]:
dfx.reset_index()

,index,ImageID,LabelName_x,XMin_x,XMax_x,YMin_x,YMax_x,LabelName_y,XMin_y,XMax_y,YMin_y,YMax_y
0,0,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
1,1,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/01y9k5,0.231250,0.712500,0.649485,0.970947
2,2,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
3,3,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
4,4,000060e3121c7305,/m/01mzpv,0.368750,0.538750,0.623243,0.999063,/m/04bcr3,0.630000,0.996875,0.685098,0.999063
5,5,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
6,6,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/01y9k5,0.231250,0.712500,0.649485,0.970947
7,7,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
8,8,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
9,9,000060e3121c7305,/m/01y9k5,0.231250,0.712500,0.649485,0.970947,/m/04bcr3,0.630000,0.996875,0.685098,0.999063


In [0]:
dfx.columns = ['ImageID','LabelName1','XMin1',
                     'XMax1','YMin1','YMax1','LabelName2','XMin2',
                     'XMax2','YMin2','YMax2']
dfx.head()

,ImageID,LabelName1,XMin1,XMax1,YMin1,YMax1,LabelName2,XMin2,XMax2,YMin2,YMax2
0,000060e3121c7305,/m/01mzpv,0.36875,0.53875,0.623243,0.999063,/m/01mzpv,0.368750,0.538750,0.623243,0.999063
1,000060e3121c7305,/m/01mzpv,0.36875,0.53875,0.623243,0.999063,/m/01y9k5,0.231250,0.712500,0.649485,0.970947
2,000060e3121c7305,/m/01mzpv,0.36875,0.53875,0.623243,0.999063,/m/01y9k5,0.629375,0.999375,0.691659,0.997188
3,000060e3121c7305,/m/01mzpv,0.36875,0.53875,0.623243,0.999063,/m/04bcr3,0.228750,0.696875,0.465792,0.926898
4,000060e3121c7305,/m/01mzpv,0.36875,0.53875,0.623243,0.999063,/m/04bcr3,0.630000,0.996875,0.685098,0.999063


In [0]:
cols = dfx.columns.tolist()

cols

['ImageID',
 'LabelName1',
 'XMin1',
 'XMax1',
 'YMin1',
 'YMax1',
 'LabelName2',
 'XMin2',
 'XMax2',
 'YMin2',
 'YMax2']

In [0]:
cols=['ImageID',
 'LabelName1',
 'LabelName2',
 'XMin1',
 'XMax1',
 'YMin1',
 'YMax1',
 'XMin2',
 'XMax2',
 'YMin2',
 'YMax2']

In [0]:
dfx = dfx[cols]
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6443509 entries, 0 to 6443508
Data columns (total 11 columns):
ImageID       object
LabelName1    object
LabelName2    object
XMin1         float64
XMax1         float64
YMin1         float64
YMax1         float64
XMin2         float64
XMax2         float64
YMin2         float64
YMax2         float64
dtypes: float64(8), object(3)
memory usage: 749.9+ MB


In [0]:
dfxf= pd.merge(df,dfx, on=['ImageID','LabelName1','XMin1',
                     'XMax1','YMin1','YMax1','LabelName2','XMin2',
                     'XMax2','YMin2','YMax2'],how='outer')

dfxf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6622838 entries, 0 to 6622837
Data columns (total 12 columns):
ImageID              object
LabelName1           object
LabelName2           object
XMin1                float64
XMax1                float64
YMin1                float64
YMax1                float64
XMin2                float64
XMax2                float64
YMin2                float64
YMax2                float64
RelationshipLabel    object
dtypes: float64(8), object(4)
memory usage: 656.9+ MB


In [0]:
rem=dfxf[ (dfxf['LabelName1'] == dfxf['LabelName2']) & (dfxf['XMin1'] == dfxf['XMin2']) & (dfxf['XMax1'] == dfxf['XMax2'])& (dfxf['YMin1'] == dfxf['YMin2']) & (dfxf['XMin1'] == dfxf['XMin2'])]
dfxf.drop(rem.index,inplace=True)

In [0]:
dfxf=dfxf[:300000]
dfxf['RelationshipLabel'].value_counts()

at                111493
on                 31604
holds              20986
plays               8932
interacts_with      3756
inside_of           2392
wears                836
hits                 593
under                 34
Name: RelationshipLabel, dtype: int64

In [0]:
dfxf.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,000060e3121c7305,/m/01mzpv,/m/04bcr3,0.36875,0.538750,0.623243,0.999063,0.22875,0.696875,0.465792,0.926898,at
1,000060e3121c7305,/m/01mzpv,/m/01y9k5,0.36875,0.538750,0.623243,0.999063,0.23125,0.712500,0.649485,0.970947,at
2,0000615b5a80f660,/m/01mzpv,/m/078n6m,0.00375,0.166250,0.663333,0.941667,0.00000,0.694375,0.778333,0.999167,at
3,0000615b5a80f660,/m/09tvcd,/m/078n6m,0.34875,0.400625,0.760000,0.913333,0.00000,0.694375,0.778333,0.999167,on
4,0000615b5a80f660,/m/01mzpv,/m/078n6m,0.00000,0.114375,0.926667,0.999167,0.00000,0.694375,0.778333,0.999167,at


In [0]:
dfxf['IOU']=np.nan

In [0]:
#def getboxA(XMin1,XMax1,YMin1,YMax1):
#  boxA=[XMin1, XMax1,	YMin1,	YMax1]
#def getboxB(XMin2,XMax2,YMin2,YMax2):
#  boxB=[XMin2, XMax2,	YMin2,	YMax2]

#def bb_intersection_over_union(XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2):

# determine the (x, y)-coordinates of the intersection rectangle
for index, row in dfxf.iterrows():
    boxA=[dfxf['XMin1'][index],dfxf['XMax1'][index],dfxf['YMin1'][index],dfxf['YMax1'][index]]
    boxB=[dfxf['XMin2'][index],dfxf['XMax2'][index],dfxf['YMin2'][index],dfxf['YMax2'][index]]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 # compute the area of both the prediction and ground-truth
	# rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 # compute the intersection over union by taking the intersection
 # area and dividing it by the sum of prediction + ground-truth
# areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 # return the intersection over union value
    dfxf['IOU'][index]=iou  
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
#dfxf['IOU']=bb_intersection_over_union(dfxf.XMin1,dfxf.XMax1,dfxf.YMin1,dfxf.YMax1,dfxf.XMin2,dfxf.XMax2,dfxf.YMin2,dfxf.YMax2)
#dfxf.head()

In [0]:
#bb_intersection_over_union(0.36875,	0.538750,	0.623243,	0.999063,0.36875,	0.538750,	0.623243,	0.999063)
#w

In [0]:
#dfxf['XMin1'],dfxf['XMax1'],dfxf['YMin1'],dfxf['YMax1'],dfxf['XMin2'],dfxf['XMax2'],dfxf['YMin2'],dfxf['YMax2']

In [0]:
dfxf.head()

In [0]:
#import pandas as pd
#import multiprocessing

# create as many processes as there are CPUs on your machine
#num_processes = multiprocessing.cpu_count()

# calculate the chunk size as an integer
#chunk_size = int(df.shape[0]/num_processes)

# this solution was reworked from the above link.
# will work even if the length of the dataframe is not evenly divisible by num_processes
#chunks = [dfxf.loc[df.index[i:i + chunk_size]] for i in range(0, dfxf.shape[0], chunk_size)]
#num_processes

In [0]:

#for index, row in dfxf.iterrows():
#    boxA=[dfxf['XMin1'][index],dfxf['XMax1'][index],dfxf['YMin1'][index],dfxf['YMax1'][index]]
#    boxB=[dfxf['XMin2'][index],dfxf['XMax2'][index],dfxf['YMin2'][index],dfxf['YMax2'][index]]
#    xA = max(boxA[0], boxB[0])
#    yA = max(boxA[1], boxB[1])
#    xB = min(boxA[2], boxB[2])
#    yB = min(boxA[3], boxB[3])
 # compute the area of intersection rectangle
#    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 # compute the area of both the prediction and ground-truth
	# rectangles
#    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 # compute the intersection over union by taking the intersection
 # area and dividing it by the sum of prediction + ground-truth
# areas - the interesection area
#    iou = interArea / float(boxAArea + boxBArea - interArea)
 # return the intersection over union value
#    dfxf['IOU'][index]=iou  
  

In [0]:
dfxf.tail(50)

In [0]:

# create our pool with `num_processes` processes
#pool = multiprocessing.Pool(processes=num_processes)

# apply our function to each chunk in the list
#result = pool.map(func, chunks)

In [0]:
#for i in range(len(result)):
   # since result[i] is just a dataframe
   # we can reassign the original dataframe based on the index of each chunk
#   dfxf.loc[result[i].index] = result[i]

In [0]:
#dfxf.head()

In [0]:
dfxf.to_pickle(dfxf) 